In [48]:
#imports
import utils.analysis as analysis
import utils.datautils as datautils
import matplotlib.pyplot as plt
import tifffile as tif
import numpy as np
import pandas as pd
from read_roi import read_roi_zip as col_zip
from read_roi import read_roi_file as col_f
import os
from PIL import Image
import seaborn as sns

In [49]:
data_path = '/Users/aichah/Desktop/TLI/data/columns/'
save_path = '/Users/aichah/Desktop/TLI/data/columns/output/'

In [50]:
files_list = datautils.get_file_names(data_path, group_by='.zip', 
                                    order=True, nested_files=False, 
                                    criteria='')

first 5 files
['/Users/aichah/Desktop/TLI/data/columns/T4_50_sD_RoiSet.zip', '/Users/aichah/Desktop/TLI/data/columns/T4_49_sD_RoiSet.zip', '/Users/aichah/Desktop/TLI/data/columns/T4_47_sC_RoiSet.zip', '/Users/aichah/Desktop/TLI/data/columns/T4_45_sA_RoiSet.zip', '/Users/aichah/Desktop/TLI/data/columns/T4_44_sD_RoiSet.zip']


In [51]:
av_col = col_f('/Users/aichah/Desktop/TLI/data/columns/avg_Col.roi')
av_roi = av_col['avg_Col']
x1 = av_roi['left']+int(av_roi['width']/2); a = int(av_roi['width']/2)  # x center, half width                                       
y1 = av_roi['top']+int(av_roi['height']/2); b = int(av_roi['height']/2)  # y center, half height
a1=av_roi['height']
b1=av_roi['width']

In [52]:
results = []

In [75]:
for file in files_list:
    name = os.path.basename(file)[:-11]
    subtype = name[-1]
    Cols = col_zip(file)
    ROI_2D = Cols['0,0']
    av_roi = av_col['avg_Col']
    x0 = ROI_2D['left']+int(ROI_2D['width']/2); a = int(ROI_2D['width']/2)  # x center, half width                                       
    y0 = ROI_2D['top']+int(ROI_2D['height']/2); b = int(ROI_2D['height']/2)  # y center, half height
    a0=ROI_2D['height']
    b0=ROI_2D['width']
    ori_y, ori_x = Cols['Entry']['y'][0],Cols['Entry']['x'][0]
    y_f, x_f = analysis.transform_point(ori_y, ori_x, 
                                    x0, y0, #center of ellipse1 
                                    a0, b0, #major and minor of ellispe1
                                    x1, y1, #center of ellipse2 (ref)
                                    a1, b1, # major and minor of ellipse2
                                    X_length=800, #X_dim of final array
                                    Y_length=800) #Y_dim of final array
    results.append([name, subtype, ori_y, ori_x, y_f, x_f])
    check_im = np.zeros([800,800])
    av_nD, av_3D = analysis.roi_img(check_im, av_col['avg_Col'])
    mask_nD, Roi_3D = analysis.roi_img(check_im, Cols['0,0'])
    Roi_3D_1 = Roi_3D.copy()
    Roi_3D_1[ori_y-2:ori_y+2, ori_x-2:ori_x+2] = 2
    final_im = av_3D + Roi_3D_1
    final_im[y_f-2:y_f+2, x_f-2:x_f+2] = 2
    final_im = 100 * final_im
    final_im = final_im.astype('uint8')
    final_im = Image.fromarray(final_im)
    final_im.save(save_path+name+'.png')

In [54]:
columns_1 = ['name', 'subtype', 'ori_y', 'ori_x', 'y_f', 'x_f']
results = pd.DataFrame(results, columns=columns_1)

In [76]:
final_im_1 = av_3D*2
final_im_2 = np.zeros((4,800,800))
for i, st in enumerate(final_im_2):
    final_im_2[i][av_3D!=0] = 2
# final_im_2 = np.broadcast_to(final_im_1, img_nD.shape)
# final_im_2 = final_im_2.copy()
factor = {'A':10, 'B':20, 'C':30, 'D':40}
colors = {'A':'red', 'B':'green', 'C':'blue', 'D':'yellow'}
for index, row in results.iterrows():
    y,x = row.y_f, row.x_f
    final_im_1[y,x] = factor[row['subtype']]
    if row.subtype == 'A':
        final_im_2[0,y-2:y+2,x-2:x+2] = 10
    elif row.subtype == 'B':
        final_im_2[1,y-2:y+2,x-2:x+2] = 10
    elif row.subtype == 'C':
        final_im_2[2,y-2:y+2,x-2:x+2] = 10
    elif row.subtype == 'D':
        final_im_2[3,y-2:y+2,x-2:x+2] = 10      

In [77]:
tif.imsave(save_path+'final_result_1.tif', final_im_1)
tif.imsave(save_path+'final_result_5.tif', final_im_2[:,:400,:400])